# 游戏环节A/B test设计及用户留存率分析

本项目通过对某款游戏的某个关卡进行A/B test实验，判断哪种方式可以吸引更多的用户留存

## 一、项目背景

本项目设计到一个益智游戏：Cookies Cats。  
随着玩家逐渐深入了解游戏，系统会为玩家们设置Gate，使他们等待一段时间或者是进行物品的交换  
除此之外，这些Gate还可以使玩家获得强制性休息以延长游戏在线时间以及增加乐趣  
但是，如果Gate放置位置不正确，可能会导致玩家产生负面情绪甚至流失  
本项目通过设计A/B test实验来帮助分析在哪一环节防止Gate可以获得更大的收益

## 二、查看数据

数据由90189行，5列数据构成，名次解释如下：
- userid: 每个玩家的独一的id
- version: 玩家的组别(Gate 30组或者Gate 40组)
- sum_gamerounds: 玩家安装游戏一周后游玩的关卡数
- retention_1: 安装1天后的留存情况
- retention_7: 安装7天后的留存情况

## 三、导入库

In [2]:
import numpy as np
import matplotlib as plt
import pandas as pd

## 四、读取数据

In [3]:
df=pd.read_csv(r'F:\Data\games\cookie_cats.csv')

## 五、查看数据

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [5]:
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


## 六、数据清洗

### 查看重复值

In [7]:
df.userid.duplicated().sum()

0

### 查看空值

In [8]:
df.isnull().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

数据不需要清洗

## 七、retention_1假设检验

In [16]:
# 查看两个方式下的retention_1数据
per1,per2=df.groupby(['version'])['retention_1'].mean()

In [17]:
per1

0.4481879194630872

In [18]:
per2

0.44228274967574577

In [19]:
print('gate_30留存率：%.20f' %per1)
print('gate_40留存率：%.20f' %per2)

gate_30留存率：0.44818791946308722540
gate_40留存率：0.44228274967574576770


我们可以看出gate_40比gate_30的结果少了0.6个百分点，运用假设检验判断是否结果正确

### 零假设/备择假设

零假设H<sub>0</sub>：p0>=p1，备择假设H<sub>1</sub>：p0<p1

### 假设检验方向

因为备择假设H<sub>1</sub>，我们选用左尾

### 检验类型和统计量

In [23]:
df.groupby('version').count()

,userid,sum_gamerounds,retention_1,retention_7
version,,,,
gate_30,44700,44700,44700,44700
gate_40,45489,45489,45489,45489


检验量都比较大，所以我们选用z检验

### 显著性水平α

我们定义α为0.05

### 实验过程

#### 导入库

In [10]:
import statsmodels.stats.proportion as sp

#### 数据准备

In [12]:
con_old=df[(df.version=='gate_30')&(df.retention_1==1)].shape[0]
con_old

20034

In [13]:
con_new=df[(df.version=='gate_40')&(df.retention_1==1)].shape[0]
con_new

20119

In [14]:
n_old=df[(df.version=='gate_30')].shape[0]
n_old

44700

In [15]:
n_new=df[(df.version=='gate_40')].shape[0]
n_new

45489

#### 数据计算

由之前可得，per1比per2多了0.6个百分点，所以这里的alternative='larger'

In [20]:
z_score, p_value = sp.proportions_ztest([con_old, con_new], [n_old, n_new], alternative='larger')

In [21]:
z_score

1.7840862247974725

In [22]:
p_value

0.03720482764845957

因为p_value=0.0372<0.05=α,我们可以判断出H<sub>0</sub>发生的概率较小，拒绝H<sub>0</sub>，接受H<sub>1</sub>

所以得出结论：gate_30比gate_40可以提升用户第二天的留存率

## 八、retention_7假设检验

由之前我们得到，当gate_30的时候，1天之后的用户留存率比较高  
由于玩家只玩了一天的游戏，可能大多数玩家没有到30级，会对实验结果有影响  
但是玩了一周之后，更多的玩家会到40级，因此我们也需要考虑用户7天后的留存率

In [26]:
# 查看两个方式下的retention_7数据
per1,per2=df.groupby(['version'])['retention_7'].mean()

In [27]:
per1

0.19020134228187918

In [28]:
per2

0.18200004396667327

In [29]:
print('gate_30留存率：%.20f' %per1)
print('gate_40留存率：%.20f' %per2)

gate_30留存率：0.19020134228187918213
gate_40留存率：0.18200004396667326945


我们可以看出gate_40比gate_30的结果少了0.8个百分点，运用假设检验判断是否结果正确

### 零假设/备择假设

零假设H<sub>0</sub>：per0>=per1，备择假设H<sub>1</sub>：per0<per1

### 假设检验方向

因为备择假设H1，我们选用左尾

### 检验类型和统计量

In [31]:
df.groupby('version').count()

,userid,sum_gamerounds,retention_1,retention_7
version,,,,
gate_30,44700,44700,44700,44700
gate_40,45489,45489,45489,45489


检验量都比较大，所以我们选用z检验

### 显著性水平α

我们定义α为0.05

### 实验过程

#### 导入库

In [32]:
import statsmodels.stats.proportion as sp

#### 数据准备

In [33]:
con_old=df[(df.version=='gate_30')&(df.retention_7==1)].shape[0]
con_old

8502

In [34]:
con_new=df[(df.version=='gate_40')&(df.retention_7==1)].shape[0]
con_new

8279

In [35]:
n_old=df[(df.version=='gate_30')].shape[0]
n_old

44700

In [37]:
n_new=df[(df.version=='gate_40')].shape[0]
n_new

45489

#### 数据计算

由之前可得，per1比per2多了0.6个百分点，所以这里的alternative='larger'

In [39]:
z_score, p_value = sp.proportions_ztest([con_old, con_new], [n_old, n_new], alternative='larger')

In [40]:
z_score

3.164358912748191

In [41]:
p_value

0.0007771249878071645

因为p_value=0.00077<0.05=α,我们可以判断出H0发生的概率较小，拒绝H0，接受H1

所以得出结论：gate_30比gate_40可以提升用户第七天的留存率

## 九、结论

A/B test实验结果告诉我们，在30级设置gate与40级设置gate相比，1天以及7天的用户留存率都会提高，所以我们更应该在30级设置gate

其实在A/B test实验之前我们可以研究更多的指标，但是用户留存率/活跃度使我们最关心的指标，基于此我们才做了这个A/B 测试

为什么30级设置gate比40级设置gate更能提高用户留存率？  
通常来说，gate相当于一个负面措施，因为用户不得不等待一段时间进行物品的交换，通常我们都认为gate放置的越后面，用户留存率应该越高  

但是这与我们实验的结果相悖，我们可以利用hedonic adaptation理论来解释  
通俗地来说，hedonic adaptation与边际效应递减很像，人们从一种活动中获得的乐趣会越来越少  
如果在30级迫使他们休息一下，反而可以延长他们的游戏乐趣  
但是如果在40级设置gate，更多的玩家会在40级之前感到游戏无趣，会更容易退出游戏